## 範例重點
* 了解如何 reset Tensorflow Graph
* 學習如何以迴圈方式訓練不同超參數的模型
* 學習如何以迴圈方式繪圖

In [1]:
import os
from tensorflow import keras

# 本範例不需使用 GPU, 將 GPU 設定為 "無"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# 從 Keras 的內建功能中，取得 train 與 test 資料集
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# 資料前處理 - X 標準化
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# 資料前處理 -Y 轉成 onehot
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(input_layer)
        else:
            x = keras.layers.Dense(units=n_units, activation="relu", name="hidden_layer"+str(i+1))(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
## 超參數設定
LEARNING_RATE = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
EPOCHS = 50
BATCH_SIZE = 2048 # 256
MOMENTUM = 0.95

In [7]:
results = {}
"""
使用迴圈，建立不同 Learning rate 的模型並訓練
"""
for lr in LEARNING_RATE:
    keras.backend.clear_session() # 把舊的 Graph 清掉
    print("Experiment with LR = %.6f" % (lr))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    optimizer = keras.optimizers.SGD(lr=lr, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True)
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["accuracy"]
    valid_acc = model.history.history["val_accuracy"]
    
    exp_name_tag = "exp-lr-%s" % str(lr)
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}


Experiment with LR = 0.100000
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
output (Dense)               (None, 10)                1290      
Total params: 1,738,890
Trainable params: 1,738,890
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [=============

50000/50000 [==============================] - 3s 65us/sample - loss: 1.2376 - accuracy: 0.5579 - val_loss: 1.5255 - val_accuracy: 0.4809
Epoch 49/50
50000/50000 [==============================] - 3s 64us/sample - loss: 1.2532 - accuracy: 0.5535 - val_loss: 1.4916 - val_accuracy: 0.4796
Epoch 50/50
50000/50000 [==============================] - 3s 65us/sample - loss: 1.1934 - accuracy: 0.5717 - val_loss: 1.5068 - val_accuracy: 0.4830
Experiment with LR = 0.010000
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3072)]            0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_______________________________________________________

50000/50000 [==============================] - 3s 66us/sample - loss: 1.0196 - accuracy: 0.6443 - val_loss: 1.4119 - val_accuracy: 0.5085
Epoch 46/50
50000/50000 [==============================] - 3s 66us/sample - loss: 1.0109 - accuracy: 0.6487 - val_loss: 1.3385 - val_accuracy: 0.5329
Epoch 47/50
50000/50000 [==============================] - 3s 66us/sample - loss: 1.0185 - accuracy: 0.6438 - val_loss: 1.3581 - val_accuracy: 0.5276
Epoch 48/50
50000/50000 [==============================] - 3s 66us/sample - loss: 0.9919 - accuracy: 0.6524 - val_loss: 1.3177 - val_accuracy: 0.5424
Epoch 49/50
50000/50000 [==============================] - 3s 65us/sample - loss: 0.9847 - accuracy: 0.6585 - val_loss: 1.4070 - val_accuracy: 0.5085
Epoch 50/50
50000/50000 [==============================] - 3s 65us/sample - loss: 0.9864 - accuracy: 0.6582 - val_loss: 1.3405 - val_accuracy: 0.5364
Experiment with LR = 0.001000
Model: "model"
_________________________________________________________________
L

50000/50000 [==============================] - 3s 65us/sample - loss: 1.5577 - accuracy: 0.4573 - val_loss: 1.5724 - val_accuracy: 0.4514
Epoch 43/50
50000/50000 [==============================] - 3s 65us/sample - loss: 1.5521 - accuracy: 0.4597 - val_loss: 1.5704 - val_accuracy: 0.4505
Epoch 44/50
50000/50000 [==============================] - 3s 66us/sample - loss: 1.5471 - accuracy: 0.4615 - val_loss: 1.5643 - val_accuracy: 0.4530
Epoch 45/50
50000/50000 [==============================] - 3s 66us/sample - loss: 1.5425 - accuracy: 0.4626 - val_loss: 1.5619 - val_accuracy: 0.4555
Epoch 46/50
50000/50000 [==============================] - 3s 66us/sample - loss: 1.5370 - accuracy: 0.4638 - val_loss: 1.5560 - val_accuracy: 0.4534
Epoch 47/50
50000/50000 [==============================] - 3s 66us/sample - loss: 1.5328 - accuracy: 0.4658 - val_loss: 1.5523 - val_accuracy: 0.4564
Epoch 48/50
50000/50000 [==============================] - 3s 66us/sample - loss: 1.5277 - accuracy: 0.4681 - va

50000/50000 [==============================] - 3s 67us/sample - loss: 1.9315 - accuracy: 0.3234 - val_loss: 1.9340 - val_accuracy: 0.3189
Epoch 40/50
50000/50000 [==============================] - 3s 65us/sample - loss: 1.9278 - accuracy: 0.3244 - val_loss: 1.9304 - val_accuracy: 0.3193
Epoch 41/50
50000/50000 [==============================] - 3s 67us/sample - loss: 1.9243 - accuracy: 0.3256 - val_loss: 1.9270 - val_accuracy: 0.3234
Epoch 42/50
50000/50000 [==============================] - 3s 66us/sample - loss: 1.9207 - accuracy: 0.3273 - val_loss: 1.9235 - val_accuracy: 0.3236
Epoch 43/50
50000/50000 [==============================] - 3s 67us/sample - loss: 1.9174 - accuracy: 0.3275 - val_loss: 1.9202 - val_accuracy: 0.3244
Epoch 44/50
50000/50000 [==============================] - 3s 65us/sample - loss: 1.9141 - accuracy: 0.3293 - val_loss: 1.9170 - val_accuracy: 0.3272
Epoch 45/50
50000/50000 [==============================] - 3s 67us/sample - loss: 1.9109 - accuracy: 0.3305 - va

50000/50000 [==============================] - 3s 65us/sample - loss: 2.2095 - accuracy: 0.2049 - val_loss: 2.2099 - val_accuracy: 0.2068
Epoch 37/50
50000/50000 [==============================] - 3s 67us/sample - loss: 2.2075 - accuracy: 0.2064 - val_loss: 2.2080 - val_accuracy: 0.2084
Epoch 38/50
50000/50000 [==============================] - 3s 66us/sample - loss: 2.2055 - accuracy: 0.2075 - val_loss: 2.2060 - val_accuracy: 0.2108
Epoch 39/50
50000/50000 [==============================] - 3s 65us/sample - loss: 2.2036 - accuracy: 0.2096 - val_loss: 2.2041 - val_accuracy: 0.2125
Epoch 40/50
50000/50000 [==============================] - 3s 65us/sample - loss: 2.2016 - accuracy: 0.2111 - val_loss: 2.2022 - val_accuracy: 0.2137
Epoch 41/50
50000/50000 [==============================] - 3s 66us/sample - loss: 2.1997 - accuracy: 0.2124 - val_loss: 2.2004 - val_accuracy: 0.2157
Epoch 42/50
50000/50000 [==============================] - 3s 66us/sample - loss: 2.1979 - accuracy: 0.2135 - va

In [8]:
import matplotlib.pyplot as plt
%matplotlib inline
color_bar = ["r", "g", "b", "y", "m", "k"]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

## Work
1. 請比較 SGD optimizer 不同的 momentum 及使用 nesterov 與否的表現